In [1]:
%pylab inline
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.util import add_cyclic_point
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib.path as mpath
import matplotlib.ticker as mticker
from matplotlib.patches import Polygon
import cmocean.cm as cmo
import gsw

Populating the interactive namespace from numpy and matplotlib


In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()

datadir = '/Users/erik/Desktop/'

In [3]:
T = xr.open_mfdataset(datadir+'GODAS/pottmp*', combine='by_coords').pottmp.sel(lat=slice(-90, -30)).groupby('time.month').mean("time").sel(month=10)
S = xr.open_mfdataset(datadir+'GODAS/salt*', combine='by_coords').salt.sel(lat=slice(-90, -30)).groupby('time.month').mean("time").sel(month=10)

tempA = np.array(T-273.15, dtype=np.float32)
saltA = np.array(S*1000, dtype=np.float32)

/Users/erik/anaconda3/envs/py3_parcels/lib/python3.6/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/Users/erik/anaconda3/envs/py3_parcels/lib/python3.6/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [4]:
dens = np.zeros((tempA.shape[0], tempA.shape[1]*tempA.shape[2]))
lat, lon = np.meshgrid(T.lat, T.lon)
lat = lat.flatten().tolist()
lon = lon.flatten().tolist()
for z in range(tempA.shape[0]):
    tempL = tempA[z, :, :].flatten().tolist()
    saltL = saltA[z, :, :].flatten().tolist()
    pres = float(T.level[z])

    for i in range(len(tempL)):
        dens[z, i] = eng.eos80_legacy_gamma_n(saltL[i], tempL[i], pres, lon[i], lat[i])

In [5]:
np.save(datadir+'GODAS/gamman', dens)